# Reproducing the Tetralogogy of Fallot Cluster Analysis 

## Notebook overview 

This notebook demonstrates the cluster analysis described by Matthieu Miossec and collaborators in the bioRxiv preprint "[Deleterious genetic variants in NOTCH1 are a major contributor to the incidence of non-syndromic Tetralogy of Fallot (ToF)](https://www.biorxiv.org/content/early/2018/04/13/300905). We reproduced the original analysis in R code in a notebook by working with Dr. Miossec. 

The notebook walks users through the steps of performing the cluster analysis on two precomputed sets of synthetic data, one with 100 samples, one with 500. The two cluster analyses demonstrate how the larger cohort (comparable to the 829 cohort in the original publication) reproduces Miossec et al.'s published results. 

## Reprodicibility overview  

This notebook is intended as an example of how to make a reproducible study by doing the analysis in an interactive and shareable Jupyter Notebook. Code cells contain the exact R code adapted for reproducing the analysis and each analysis step is documented with accompanying markdown text to explain the researcher's reasoning and methods. 

## Cluster Analysis description

Given a large set of cases, variants contributing to a disease trait will form detectable clusters within specific regions of a gene or genes which, when mutated, lead to the disease. We can uncover these clusters using the WD statistic and reliable sequencing data.

For each gene coding sequence (CDS), we enter the number of hits that fall within the CDS and calculate the probability of a single hit falling within this particular CDS given the exome's total CDS size.

## Cluster Analysis steps

* Classify nonsense and frameshift mutations
* Filter based on CADD > 20, [Combined Annotation Dependent Depletion (CADD)](https://cadd.gs.washington.edu/) method
* Identify likely deleterious variant clusters in patient genes
* Generate statistics on gene clustering

| Steps                                      | Input                                           | Output |
| -------------------------------------------|-------------------------------------------------|--------|
| Filter CADD >20 | gemini_out.txt                 | gemini_filtered     |
| Count variants per gene                    |gemini_filtered                                  | variants_per_gene   |
| Add genes that have a variant count of 0 or X | variants_per_gene | variants_per_CDS |
| Sort and add CDS length as fraction of total CDS length | variants_per_CDS | variants_per_CDS_withsizes
| Cluster analysis                           | variants_per_CDS_withsizes | new_data     |



## Resources used in the original analysis

* [Combined Annotation Dependent Depletion](https://cadd.gs.washington.edu/) (CADD) [v.1.3](https://cadd.gs.washington.edu/static/ReleaseNotes_CADD_v1.3.pdf)
* [GoogleCloudStorageR](https://github.com/cloudyr/googleCloudStorageR) v.0.4.0
* [Ensembl 75](http://feb2014.archive.ensembl.org/index.html) provides gene coding sequence sizes

# Set the notebook environment

## Install Google Cloud's R package

In [ ]:
install.packages("googleCloudStorageR")

## Generate a key to access Gemini outputs in the workspace's Google bucket

In [ ]:
key <- Ronaldo::getServiceAccountKey()

## Save the key and write it to a json file that can be accessed from your local cluster

In [ ]:
fileConn<-file("/home/jupyter/key.json")
writeLines(key, fileConn)
close(fileConn)

## Set environment variable so that googleCloudStorageR can authenticate itself

In [ ]:
Sys.setenv("GCS_AUTH_FILE" = "/home/jupyter/key.json")

In [ ]:
library(googleCloudStorageR)

### To see more features:  <a class="tocSkip">

Enter the command `library(help = googleCloudStorageR)` and execute in the code cell (below) 

# Read in and filter data

In this section, we read the output from the **6_Predict-variant-effects-GEMINI** WDL and start filtering. 

We capture likely deleterious and missense variants by first filtering out variants that are not stop mutations or frameshift. Then we weed out the missense variants using a CADD score >=20. What's left are likely deleterious variants.

**Note:** This notebook reproduces and compares two analyses: one with a participant set of 100 and a second with a particpant set of 500. Because generating and alalyzing a 500-sample synthetic cohort is computationally expensive and long, the notebook will offer two options:           
1. Use precomputed files from `6_predict-variant-effects-GEMINI`    
2. Analyzing a pre-computed 500-sample cohort 

## Analyze 100-sample synthetic cohort

### Read gemini_out.txt from the Google bucket and save it to the local Jupyter cluster

In [ ]:
gcs_get_object("gs://firecloud-workshops/181017-ashg18/predicted-effects/Test-cohort-A100.jointcalls.filtered.gemini-predictions.txt", 
               saveToDisk = "/home/jupyter/notebooks/gemini_out.txt",overwrite= TRUE)

gemini_out <- read.table("/home/jupyter/notebooks/gemini_out.txt",sep = "\t", header = TRUE)

### Filter for CADD score >= 20.

In [ ]:
gemini_out$cadd_scaled <- as.numeric(as.character(gemini_out$cadd_scaled))
gemini_out$cadd_scaled[is.na(gemini_out$cadd_scaled)] <- 0   

# Note that the warning message about "NAs introduced by coercion" is expected. The function eliminates 
# zeros in the data set and doesn't impact the results.

In [ ]:
gemini_filtered <- gemini_out[(gemini_out$biotype=="protein_coding")& (gemini_out$cadd_scaled>=20),]

### Read ensemble gene CDS sizes from a file in the Google bucket

In [ ]:
gcs_get_object("gs://firecloud-workshops/181017-ashg18/annotation/Ensembl75_gene_CDS_sizes.txt",
               saveToDisk = "/home/jupyter/notebooks/ENS75.txt",overwrite= TRUE)
CDS_size <- read.table("/home/jupyter/notebooks/ENS75.txt",sep = "\t", header = TRUE)

### Count number of variants per gene

In [ ]:
variants_per_gene<-aggregate(x = gemini_filtered$gene, by = list(gene = gemini_filtered$gene), FUN = length)
colnames(variants_per_gene) <- c("gene","variant_count")

### Add genes that have a variant count of zero and remove genes that don't have an incomplete CDS

In [ ]:
variants_per_CDS<-rbind.data.frame(variants_per_gene[variants_per_gene$gene %in% CDS_size$gene,], data.frame(gene = setdiff(CDS_size$gene,variants_per_gene$gene), variant_count = 0))                   


### Sort and add CDS length as fraction of total CDS length

We want to compare the number of variants we would expect by chance and the number of variants we actually observe. Some genes are so big that they’ll definitely have more of variants than NOTCH1. What’s significant is how many variants a gene has given its size. 

In [ ]:
sorted <- variants_per_CDS[order(as.character(variants_per_CDS$gene)),]
variants_per_CDS_withsizes <- data.frame(sorted,length_proportion=CDS_size$length_proportion)

# Print the variants per length
variants_per_CDS_withsizes

### Identify genes where significantly more variants are observed than expected (Cluster.Test method)

The method takes in two arguments: NbrInTarget and TargetProbability:

* **NbrInTarget** is derived from our data and consists of a list of the number of variants (or hits) per each CDS

* **TargetProbability** is the fraction of the total human coding sequence represented by each gene coding sequence. (i.e. gene CDS size/total CDS size). This information is typically derived from BioMart but we can just provide it directly in a file in the Google bucket.

Therefore:
* N is the number of genes/gene CDS being tested
* TotalHits is the total number of variants
* ExpectedHits is the number of variants we expect to see in a gene CDS based on its size

The last line applies multiple testing correction to the statistical method (i.e. N tests for N gene CDS).


In [ ]:
Cluster.Test <- function(NbrInTarget,TargetProbability){
N <- length(NbrInTarget)
TotalHits <- sum(NbrInTarget)
ExpectedHits <- TotalHits*TargetProbability
# N*ppois(NbrInTarget,ExpectedHits,lower=F)
1-exp(ppois(NbrInTarget,ExpectedHits,lower=T,log=T)*N)
}

### Review the output of the Cluster.Test

In [ ]:
newdata <- data.frame(variants_per_CDS_withsizes, cluster.test.result = Cluster.Test(variants_per_CDS_withsizes$variant_count,variants_per_CDS_withsizes$length_proportion))
newdata[order(newdata$cluster.test.result),]

In [ ]:
Variant_counts <- head(newdata[order(newdata$cluster.test.result),],20)
Variant_counts$minuslog10p = -log10(Variant_counts$cluster.test.result+1e-100)
bars <- barplot(Variant_counts$minuslog10p,names.arg = Variant_counts$gene, las=2)

Notice that NOTCH1 does harbour an excess of rare, deleterious variants among ToF patients in the clustering analysis. 

However, the NOTCH1 variants were not the top hit found in the original ToF study. Our hypothesis is that the small cohort test run was statistically underpowered.

**Let's repeat the analysis with the 500-sample synthetic cohort to test this hypothesis...**

## Analyze 500-sample synthetic cohort

The next series of cells repeats the cluster analysis above, this time on data from the 500-sample synthetic cohort (precomputed).

### Read gemini_out.txt from the Google bucket and save it to the local Jupyter cluster

In [ ]:
gcs_get_object("gs://firecloud-workshops/181017-ashg18/predicted-effects/Test-cohort-A500.jointcalls.filtered.gemini-predictions.txt",
               saveToDisk = "/home/jupyter/notebooks/gemini_out.txt",overwrite= TRUE)
gemini_out<-read.table("/home/jupyter/notebooks/gemini_out.txt",sep = "\t", header = TRUE)

### Filter for CADD score >= 20

In [ ]:
gemini_out$cadd_scaled <- as.numeric(as.character(gemini_out$cadd_scaled))
gemini_out$cadd_scaled[is.na(gemini_out$cadd_scaled)] <- 0

In [ ]:
gemini_filtered <- gemini_out[(gemini_out$biotype=="protein_coding")& (gemini_out$cadd_scaled>=20),]

### Read ensemble gene CDS sizes from a file in the Google bucket

In [ ]:
gcs_get_object("gs://firecloud-workshops/181017-ashg18/annotation/Ensembl75_gene_CDS_sizes.txt",
               saveToDisk = "/home/jupyter/notebooks/ENS75.txt",overwrite= TRUE)
CDS_size <- read.table("/home/jupyter/notebooks/ENS75.txt",sep = "\t", header = TRUE)

### Count number of variants per gene

In [ ]:
variants_per_gene <- aggregate(x = gemini_filtered$gene, by = list(gene = gemini_filtered$gene), FUN = length)
colnames(variants_per_gene) <- c("gene","variant_count")

### Add genes that have a variant count of zero and remove genes that don't have an incomplete CDS

In [ ]:
variants_per_CDS <- rbind.data.frame(variants_per_gene[variants_per_gene$gene %in% CDS_size$gene,], data.frame(gene = setdiff(CDS_size$gene,variants_per_gene$gene), variant_count = 0))                   


### Sort and add CDS length as fraction of total CDS length

We want to compare the number of variants we would expect by chance and the number of variants we actually observe. Some genes are so big that they’ll definitely have more of variants than NOTCH1. What’s significant is how many variants a gene has given its size. 

In [ ]:
sorted <- variants_per_CDS[order(as.character(variants_per_CDS$gene)),]
variants_per_CDS_withsizes <- data.frame(sorted,length_proportion=CDS_size$length_proportion)

# Print out results to check
variants_per_CDS_withsizes

### Identify genes where significantly more variants are observed than expected (Cluster.Test method)

The method takes in two arguments: NbrInTarget and TargetProbability:

* **NbrInTarget** is derived from our data and consists of a list of the number of variants (or hits) per each CDS.

* **TargetProbability** is the fraction of the total human coding sequence represented by each gene coding sequence. (i.e. gene CDS size/total CDS size). This information is typically derived from BioMart but we can just provide it directly in a file in the Google bucket.

Therefore:
* N is the number of genes/gene CDS being tested
* TotalHits is the total number of variants
* ExpectedHits is the number of variants we expect to see in a gene CDS based on its size

The last line applies multiple testing correction to the statistical method (i.e. N tests for N gene CDS).


In [ ]:
Cluster.Test <- function(NbrInTarget,TargetProbability){
N <- length(NbrInTarget)
TotalHits <- sum(NbrInTarget)
ExpectedHits <- TotalHits*TargetProbability
# N*ppois(NbrInTarget,ExpectedHits,lower=F)
1-exp(ppois(NbrInTarget,ExpectedHits,lower=T,log=T)*N)
}

### Review the output of the Cluster.Test

In [ ]:
newdata <- data.frame(variants_per_CDS_withsizes, cluster.test.result = Cluster.Test(variants_per_CDS_withsizes$variant_count,variants_per_CDS_withsizes$length_proportion))
newdata[order(newdata$cluster.test.result),]

In [ ]:
Variant_counts <- head(newdata[order(newdata$cluster.test.result),],20)
Variant_counts$minuslog10p = -log10(Variant_counts$cluster.test.result+1e-100)
bars <- barplot(Variant_counts$minuslog10p,names.arg = Variant_counts$gene, las=2)

Notice in particular the ranking of deleterious variants for the 500-sample cohort. 

# Compare with original paper results

Most important to note is the difference between results when analyzing the synthetic cohorts of 100 and 500 participants. The results are presented side-by-side below:     

4.1. First the original paper results alongside the results for the synthetic cohort of 100 participants 

4.2. The original paper results alongside the results for the synthetic cohort of 500 participants 

**To recap:**   
Running the cluster analysis on the 100-sample synthetic cohort identifies NOTCH1 as an important (but not **the most important**) source of deleterious variants. 

Running the cluster analysis on the 500-sample synthetic cohort, on the other hand, reveals that NOTCH1 harbours **the highest excess of rare, deleterious variants among ToF patients**.  
    
The 500 cohort reproduces the results of the original paper, which was known for its large cohort size, demonstrating the importance of large numbers for statistical analysis

## Original results versus the synthetic cohort of 100 participants 

| Original Analysis (cohort of 929) | Reproduced Analysis (synthetic cohort of 100 |    
|-----------------------------------|----------------------------------------------|     
| ![Original results from published paper](https://storage.cloud.google.com/terra-featured-workspaces/Reproducibility_Case_Study_Tetralogy_of_Fallot/ToF_Original_paper_cluster_analysis_results.png) | ![Reproduced_analysis_synthetic_cohort_of_100](https://storage.cloud.google.com/terra-featured-workspaces/Reproducibility_Case_Study_Tetralogy_of_Fallot/Synthetic_cohort_100_results_cluster_analysis_small.png) | 

## Original results versus the synthetic cohort of 500 participants

| Original analysis results (cohort of 929) | Reproduced Analysis (synthetic cohort of 500) |    
| ------------------------------------------|-----------------------------------------------|    
| ![Original results from published paper](https://storage.cloud.google.com/terra-featured-workspaces/Reproducibility_Case_Study_Tetralogy_of_Fallot/ToF_Original_paper_cluster_analysis_results.png) | ![Reproduced_analysis_synthetic_cohort_of_500](https://storage.cloud.google.com/terra-featured-workspaces/Reproducibility_Case_Study_Tetralogy_of_Fallot/Synthetic_cohort_500_cluster_analysis_results_small.png)| 